In [1]:
import sys
sys.path.append("../..")

import time
from time import sleep
from datetime import datetime
import threading

import MetaTrader5 as mt5
import pandas as pd
from IPython.display import display
from lightweight_charts import JupyterChart
from IPython.display import clear_output, display

from sesto.fractal import high_low_finder
from sesto.utils import get_price_at_pnl, calculate_fee, calculate_position_size, get_pnl_at_price, convert_lots_to_usd
from sesto.metatrader.business import get_positions, send_market_order, modify_sl_tp
from sesto.metatrader.data import fetch_data_pos
from sesto.constants import MT5Timeframe

MetaTrader5 initialized successfully
MetaTrader 5 initialized successfully.


In [2]:
# PAIRS = ['NZDJPY', 'AUDJPY', 'CADJPY', 'USDJPY']
PAIRS = ['BITCOIN', 'ETHEREUM', 'SOLANA', 'DOGECOIN', 'LITECOIN', 'RIPPLE', 'BNB', 'UNISWAP', 'AVALANCH']
MAIN_TIMEFRAME = MT5Timeframe.M15

TP_PNL_MULTIPLIER = 0.5
SL_PNL_MULTIPLIER = -0.25
LEVERAGE = 500
DEVIATION = 20
VOLUME = 0.1

TRAILING_STOP_STEPS = [
    {'trigger_pnl_multiplier': 4.00, 'new_sl_pnl_multiplier': 3.50},
    {'trigger_pnl_multiplier': 3.50, 'new_sl_pnl_multiplier': 3.00},
    {'trigger_pnl_multiplier': 3.00, 'new_sl_pnl_multiplier': 2.75},
    {'trigger_pnl_multiplier': 2.75, 'new_sl_pnl_multiplier': 2.50},
    {'trigger_pnl_multiplier': 2.50, 'new_sl_pnl_multiplier': 2.25},
    {'trigger_pnl_multiplier': 2.25, 'new_sl_pnl_multiplier': 2.00},
    {'trigger_pnl_multiplier': 2.00, 'new_sl_pnl_multiplier': 1.75},
    {'trigger_pnl_multiplier': 1.75, 'new_sl_pnl_multiplier': 1.50},
    {'trigger_pnl_multiplier': 1.50, 'new_sl_pnl_multiplier': 1.25},
    {'trigger_pnl_multiplier': 1.25, 'new_sl_pnl_multiplier': 1.00},
    {'trigger_pnl_multiplier': 1.00, 'new_sl_pnl_multiplier': 0.75},
    {'trigger_pnl_multiplier': 0.75, 'new_sl_pnl_multiplier': 0.45},
    {'trigger_pnl_multiplier': 0.50, 'new_sl_pnl_multiplier': 0.22},
    {'trigger_pnl_multiplier': 0.25, 'new_sl_pnl_multiplier': 0.12},
    {'trigger_pnl_multiplier': 0.12, 'new_sl_pnl_multiplier': 0.05},
    {'trigger_pnl_multiplier': 0.06, 'new_sl_pnl_multiplier': 0.025},
]

In [3]:
def calculate_trade_capital(symbol, volume_lots, leverage, price_open):
    position_size_usd = convert_lots_to_usd(symbol, volume_lots, price_open)
    capital_used = position_size_usd / leverage
    return capital_used

In [4]:
def have_open_positions_in_symbol(symbol):
    positions = get_positions()
    return symbol in positions['symbol'].values

In [5]:
# dict to store trades, keys are position tickets, values are the entire position object
trades = {}

In [6]:
# Utility Functions
def calculate_trade_capital(symbol, volume_lots, leverage, price_open):
    position_size_usd = convert_lots_to_usd(symbol, volume_lots, price_open)
    capital_used = position_size_usd / leverage
    return capital_used

def have_open_positions_in_symbol(symbol):
    positions = get_positions()
    return symbol in positions['symbol'].values

# Trading Logic Functions
def check_fractal_signals():
    while True:
        for pair in PAIRS:
            if have_open_positions_in_symbol(pair):
                continue

            df = fetch_data_pos(pair, MAIN_TIMEFRAME, 5)
            df['fractal'] = high_low_finder(df)
            df['symbol'] = pair

            price = mt5.symbol_info_tick(pair).ask
            new_trade_capital = calculate_trade_capital(pair, VOLUME, LEVERAGE, price)
            trade_size = calculate_position_size(new_trade_capital, LEVERAGE)
            fee = calculate_fee(position_size_usd=trade_size) 

            last_row = df.iloc[-1]
            current_time = datetime.now().replace(microsecond=0)


            if df['fractal'].iloc[-1] == 'top':
                print(f"{current_time} - {pair} - ENTRY CONDITION - TOP FRACTAL DETECTED - CLOSE: ${last_row['close']:.3f}")
                sl = get_price_at_pnl(pnl_multiplier=SL_PNL_MULTIPLIER, order_fee=fee, position_size_usd=trade_size, leverage=LEVERAGE, entry_price=price, type='short')
                order = send_market_order(symbol=pair, volume=VOLUME, order_type='sell', sl=sl, deviation=DEVIATION)
                print(f'{current_time} - {pair} - OPENED POSITION - SHORT - PRICE: {price:.3f} - SL: {sl:.3f}')
            elif df['fractal'].iloc[-1] == 'bottom':
                print(f"{current_time} - {pair} - ENTRY CONDITION - BOTTOM FRACTAL DETECTED - CLOSE: ${last_row['close']:.3f}")
                sl = get_price_at_pnl(pnl_multiplier=SL_PNL_MULTIPLIER, order_fee=fee, position_size_usd=trade_size, leverage=LEVERAGE, entry_price=price, type='long')
                order = send_market_order(symbol=pair, volume=VOLUME, order_type='buy', sl=sl, deviation=DEVIATION)
                print(f'{current_time} - {pair} - OPENED POSITION - LONG - PRICE: {price:.3f} - SL: {sl:.3f}')
            # else:
                # print(f'{current_time} - {pair} - NO FRACTAL DETECTED')

        # Wait until the next 15-minute interval
        now = current_time
        next_run = (now.replace(second=0, microsecond=0) + pd.Timedelta(minutes=15 - now.minute % 15)).replace(second=0, microsecond=0)
        next_run += pd.Timedelta(seconds=2)  # Add 2 seconds

        sleep_time = (next_run - now).total_seconds()
        print(f"Waiting for {sleep_time} seconds until next 15-minute check.")
        sleep(sleep_time)

def monitor_open_trades():
    while True:
        current_time = datetime.now().replace(microsecond=0)

        positions = get_positions()
        # print(f'{current_time} - There are {len(positions)} positions open')
        positions['time'] = pd.to_datetime(positions['time'], unit='s')
        positions['time_update'] = pd.to_datetime(positions['time_update'], unit='s')

        for index, position in positions.iterrows():
            # Check if the position ticket exists in trades dict
            if position.ticket not in trades:
                trades[position.ticket] = position
            else:
                # Check if position has changed
                if not trades[position.ticket].equals(position):
                    trades[position.ticket] = position
            
            
            # Calculate the actual capital used for this trade
            trade_capital = calculate_trade_capital(position.symbol, position.volume, LEVERAGE, position.price_open)
            trade_size = calculate_position_size(trade_capital, LEVERAGE)
            fee = calculate_fee(position_size_usd=trade_size) 
            current_pnl_percentage = (position.profit / trade_capital) * 100
            current_sl_pnl = get_pnl_at_price(position.sl, position.price_open, trade_size, LEVERAGE, 'long' if position.type == 0 else 'short')
            current_sl_pnl_percentage = (current_sl_pnl / trade_capital) * 100
            # print(f'{position.time_update} - {position.symbol} - POSITION INFO - CURRENT PRICE: ${position.price_current:.3f} - OPEN PRICE: ${position.price_open:.3f} - TRADE CAPITAL: ${trade_capital:.3f} - CURRENT SL: ${position.sl:.3f}')
            # print(f'{position.time_update} - {position.symbol} - PNL INFO - PNL: ${position.profit:.3f} or {current_pnl_percentage:.3f}% - PNL AT CURRENT SL: ${current_sl_pnl:.3f} or {current_sl_pnl_percentage:.3f}%')

            for trailing_step in TRAILING_STOP_STEPS:
                trigger_pnl_multiplier = trailing_step['trigger_pnl_multiplier']
                new_sl_pnl_multiplier = trailing_step['new_sl_pnl_multiplier']

                price_at_trigger_pnl_multiplier = get_price_at_pnl(
                    pnl_multiplier=trigger_pnl_multiplier,
                    order_fee=fee,
                    position_size_usd=trade_size,
                    leverage=LEVERAGE,
                    entry_price=position.price_open,
                    type='long' if position.type == 0 else 'short'
                )
                new_sl_at_new_sl_pnl_multiplier = get_price_at_pnl(
                    pnl_multiplier=new_sl_pnl_multiplier,
                    order_fee=fee,
                    position_size_usd=trade_size,
                    leverage=LEVERAGE,
                    entry_price=position.price_open,
                    type='long' if position.type == 0 else 'short'
                )
                pnl_at_trigger = get_pnl_at_price(
                    price_at_trigger_pnl_multiplier,
                    position.price_open,
                    trade_size,
                    LEVERAGE,
                    'long' if position.type == 0 else 'short'
                )
                pnl_at_new_sl = get_pnl_at_price(
                    new_sl_at_new_sl_pnl_multiplier,
                    position.price_open,
                    trade_size,
                    LEVERAGE,
                    'long' if position.type == 0 else 'short'
                )

                pnl_threshold = trigger_pnl_multiplier * trade_capital
                # print(f'{position.time_update} - {position.symbol} - CHECKING TRAILING SL STEP - TRIGGER: {trigger_pnl_multiplier:.3f} (${price_at_trigger_pnl_multiplier:.3f}) (${pnl_at_trigger:.3f} PNL) - NEW SL: {new_sl_pnl_multiplier:.3f} (${new_sl_at_new_sl_pnl_multiplier:.3f}) (${pnl_at_new_sl:.3f} PNL)')

                if position.profit >= pnl_threshold:
                    old_sl_price = position.sl

                    if position.type == 0:  # Long
                        new_sl_price = get_price_at_pnl(
                            pnl_multiplier=new_sl_pnl_multiplier,
                            order_fee=fee,
                            position_size_usd=trade_capital,
                            leverage=LEVERAGE,
                            entry_price=position.price_open,
                            type='long'
                        )
                        if new_sl_price > old_sl_price:
                            print(f'{position.time_update} - {position.symbol} - NEW SL: ${new_sl_price:.3f}')
                            modify_sl_tp(position.ticket, new_sl_price, position.tp)
                    else:  # Short
                        new_sl_price = get_price_at_pnl(
                            pnl_multiplier=new_sl_pnl_multiplier,
                            order_fee=fee,
                            position_size_usd=trade_capital,
                            leverage=LEVERAGE,
                            entry_price=position.price_open,
                            type='short'
                        )
                        if new_sl_price < old_sl_price:
                            print(f'{position.time_update} - {position.symbol} - NEW SL: ${new_sl_price:.3f}')
                            modify_sl_tp(position.ticket, new_sl_price, position.tp)
                    
                    break        

        # Loop through our trades to see if they are not present in position meaning they have been closed
        for trade_ticket, trade in trades.items():
            if trade_ticket not in positions['ticket'].values:
                print(f'{datetime.now()} - {trade.symbol} - POSITION CLOSED - TICKET: {trade_ticket} - PNL: ${trade.profit:.3f}')
                del trades[trade_ticket]

        sleep(10)

In [7]:
# Starting Threads
if __name__ == "__main__":
    fractal_thread = threading.Thread(target=check_fractal_signals, daemon=True)
    monitor_thread = threading.Thread(target=monitor_open_trades, daemon=True)

    fractal_thread.start()
    monitor_thread.start()

    try:
        while True:
            sleep(1)
    except KeyboardInterrupt:
        print("Program terminated by user.")

2024-10-02 18:18:32 - There are 1 positions open
Waiting for 690.0 seconds until next 15-minute check.
2024-10-02 18:18:42 - There are 1 positions open
2024-10-02 18:18:52 - There are 1 positions open
2024-10-02 18:19:02 - There are 1 positions open
2024-10-02 18:19:12 - There are 1 positions open
2024-10-02 18:19:22 - There are 1 positions open
2024-10-02 18:19:32 - There are 1 positions open
2024-10-02 18:19:42 - There are 1 positions open
2024-10-02 18:19:52 - There are 1 positions open
2024-10-02 18:20:02 - There are 1 positions open
2024-10-02 18:20:12 - There are 1 positions open
2024-10-02 18:20:22 - There are 1 positions open
2024-10-02 18:20:32 - There are 1 positions open
2024-10-02 18:20:42 - There are 1 positions open
2024-10-02 18:20:52 - There are 1 positions open
2024-10-02 18:21:02 - There are 1 positions open
2024-10-02 18:21:12 - There are 1 positions open
2024-10-02 18:21:22 - There are 1 positions open
2024-10-02 18:21:32 - There are 1 positions open
2024-10-02 18:2